In [26]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime as dt
from datetime import timedelta

In [27]:
torben_df = pd.read_csv('ac_activities_torben_clean.csv')
torben_df

,id,type,start_date_local,x_start_h,name,x_min,x_mi,x_min/mi,x_mph,x_max_mph,...,athlete_count,average_temp,distance,elapsed_time,start_latitude,start_longitude,timezone,x_date,x_elev_%,x_url
0,868156484,Run,15.01.2017 06:17:41,6.3,Morning Run,248.2,26.510,9.36,6.41,9.17,...,29,NaN,42663.3,14892,33.176906,-117.335854,(GMT-08:00) America/Los_Angeles,2017-01-15,0.51,https://www.strava.com/activities/868156484
1,868156415,Run,17.01.2017 18:50:18,18.8,Evening Run,16.1,2.023,7.95,7.55,11.41,...,30,NaN,3256.2,1300,32.720301,-117.150205,(GMT-08:00) America/Los_Angeles,2017-01-17,NaN,https://www.strava.com/activities/868156415
2,868156459,Run,19.01.2017 06:35:12,6.6,Morning Run,87.0,10.091,8.62,6.96,9.62,...,1,NaN,16240.6,5271,32.853794,-116.960111,(GMT-08:00) America/Los_Angeles,2017-01-19,0.85,https://www.strava.com/activities/868156459
3,868156435,Run,21.01.2017 07:17:12,7.3,Morning Run,88.8,11.518,7.71,7.78,9.17,...,2,NaN,18535.6,5804,32.758368,-117.242017,(GMT-08:00) America/Los_Angeles,2017-01-21,0.16,https://www.strava.com/activities/868156435
4,868156458,Run,22.01.2017 16:50:51,16.8,Afternoon Run,89.6,10.066,8.90,6.74,8.50,...,1,NaN,16199.8,5658,32.853570,-116.960471,(GMT-08:00) America/Los_Angeles,2017-01-22,0.48,https://www.strava.com/activities/868156458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,4244316603,Run,25.10.2020 11:27:04,11.5,Lunch Run,156.5,8.428,18.56,3.23,5.82,...,2,19.0,13564.3,10870,37.630499,-112.163806,(GMT-07:00) America/Denver,2020-10-25,3.91,https://www.strava.com/activities/4244316603
523,4250148881,Run,26.10.2020 15:12:37,15.2,Afternoon Run,50.3,1.887,26.64,2.25,4.25,...,1,22.0,3037.5,3212,38.309410,-109.856852,(GMT-07:00) America/Denver,2020-10-26,2.14,https://www.strava.com/activities/4250148881
524,4251737544,Run,27.10.2020 10:34:34,10.6,Morning Run,35.7,2.007,17.77,3.38,12.30,...,2,20.0,3229.2,2199,38.784718,-109.597574,(GMT-07:00) America/Denver,2020-10-27,2.04,https://www.strava.com/activities/4251737544
525,4260062116,Run,29.10.2020 13:34:15,13.6,Afternoon Run,13.4,0.717,18.67,3.21,6.04,...,1,23.0,1154.3,1874,44.725716,-110.703696,(GMT-07:00) America/Denver,2020-10-29,2.77,https://www.strava.com/activities/4260062116


In [28]:
garmin_df = pd.read_csv('ac_activities_garmin_clean.csv')
garmin_df

,Activity Type,Date,Title,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,Avg Run Cadence,Max Run Cadence,Avg Pace,Best Pace,Elev Gain,Elev Loss,Avg Stride Length,Climb Time,Min Temp,Best Lap Time,Number of Laps
0,Running,10/29/2020 14:50,Park County Running,1.02,183,0:39:41,96,127,1.0,49,158,39:03:00,3:14,276,259,0.83,39:41:00,64.4,00:02.5,2
1,Running,10/29/2020 13:34,Park County Running,0.72,137,0:31:13,91,117,0.6,40,162,43:33:00,12:53,105,62,0.91,31:13:00,68.0,31:13.0,1
2,Running,10/27/2020 10:34,Grand County Running,2.01,224,0:35:41,110,138,1.7,91,250,17:46,5:22,217,207,1.00,35:41:00,66.2,00:06.0,3
3,Running,10/26/2020 15:12,San Juan County Running,1.89,259,0:50:17,104,129,0.9,71,225,26:38:00,13:49,213,184,0.86,50:17:00,66.2,22:53.0,2
4,Running,10/25/2020 11:27,Garfield County Running,8.43,1251,3:01:06,130,178,3.7,97,250,21:30,10:07,1739,1742,0.77,3:01:06,60.8,07:42.4,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,Hiking,1/24/2015 8:07,White Water Running,15.88,1290,7:57:57,0,0,0.0,68,247,30:06:00,7:06,2207,2550,0.79,7:57:57,0.0,00:00.0,0
895,Running,1/23/2015 6:05,Santee Running,2.02,259,0:18:01,0,0,0.0,172,179,8:56,7:36,39,35,1.05,18:01,0.0,00:00.0,0
896,Running,1/21/2015 8:32,Santee Running,3.02,393,0:26:12,0,0,0.0,167,189,8:40,6:35,83,73,1.11,26:12:00,0.0,00:00.0,0
897,Running,1/20/2015 8:53,Santee Running,4.74,619,0:39:00,0,0,0.0,164,181,8:14,2:47,66,67,1.20,39:00:00,0.0,00:00.0,0


In [29]:
garmin_df.columns

Index(['Activity Type', 'Date', 'Title', 'Distance', 'Calories', 'Time',
       'Avg HR', 'Max HR', 'Aerobic TE', 'Avg Run Cadence', 'Max Run Cadence',
       'Avg Pace', 'Best Pace', 'Elev Gain', 'Elev Loss', 'Avg Stride Length',
       'Climb Time', 'Min Temp', 'Best Lap Time', 'Number of Laps'],
      dtype='object')

In [44]:
garmin_df.columns =['activity','date','title','distance','calories','time','avg_hr','max_hr','aerobic','avg_run','max_run','avg_pace','best_pace','elev_gain','elev_loss','avg_stride','climb_time','min_temp','best_lap','laps']

In [45]:
garmin_df.columns

Index(['activity', 'date', 'title', 'distance', 'calories', 'time', 'avg_hr',
       'max_hr', 'aerobic', 'avg_run', 'max_run', 'avg_pace', 'best_pace',
       'elev_gain', 'elev_loss', 'avg_stride', 'climb_time', 'min_temp',
       'best_lap', 'laps'],
      dtype='object')

In [50]:
torben_df.columns

Index(['id', 'type', 'start_date_local', 'x_start_h', 'name', 'x_min', 'x_mi',
       'x_min/mi', 'x_mph', 'x_max_mph', 'total_elevation_gain',
       'average_heartrate', 'max_heartrate', 'average_cadence',
       'x_nearest_city_start', 'x_dist_start_end_km', 'start_latlng',
       'end_latlng', 'elev_low', 'elev_high', 'athlete_count', 'average_temp',
       'distance', 'elapsed_time', 'start_latitude', 'start_longitude',
       'timezone', 'x_date', 'x_elev_%', 'x_url'],
      dtype='object')

In [57]:
torben_df.columns = ['id','type','start_date','start_h','name','x_min','x_mi','x_minpm','x_mph','x_max_mph','total_gain','avg_hr','max_hr','avg_cadence','city_start','distance_start_end','start_latlng','end_latlng','elev_low','elev_high','athlete_count','avg_temp','distance','elapsed_time','start_latitude','start_longitude','timezone','x_date','x_elev','x_url']

In [58]:
torben_df.head()

,id,type,start_date,start_h,name,x_min,x_mi,x_minpm,x_mph,x_max_mph,...,athlete_count,avg_temp,distance,elapsed_time,start_latitude,start_longitude,timezone,x_date,x_elev,x_url
0,868156484,Run,15.01.2017 06:17:41,6.3,Morning Run,248.2,26.510,9.36,6.41,9.17,...,29,NaN,42663.3,14892,33.176906,-117.335854,(GMT-08:00) America/Los_Angeles,2017-01-15,0.51,https://www.strava.com/activities/868156484
1,868156415,Run,17.01.2017 18:50:18,18.8,Evening Run,16.1,2.023,7.95,7.55,11.41,...,30,NaN,3256.2,1300,32.720301,-117.150205,(GMT-08:00) America/Los_Angeles,2017-01-17,NaN,https://www.strava.com/activities/868156415
2,868156459,Run,19.01.2017 06:35:12,6.6,Morning Run,87.0,10.091,8.62,6.96,9.62,...,1,NaN,16240.6,5271,32.853794,-116.960111,(GMT-08:00) America/Los_Angeles,2017-01-19,0.85,https://www.strava.com/activities/868156459
3,868156435,Run,21.01.2017 07:17:12,7.3,Morning Run,88.8,11.518,7.71,7.78,9.17,...,2,NaN,18535.6,5804,32.758368,-117.242017,(GMT-08:00) America/Los_Angeles,2017-01-21,0.16,https://www.strava.com/activities/868156435
4,868156458,Run,22.01.2017 16:50:51,16.8,Afternoon Run,89.6,10.066,8.90,6.74,8.50,...,1,NaN,16199.8,5658,32.853570,-116.960471,(GMT-08:00) America/Los_Angeles,2017-01-22,0.48,https://www.strava.com/activities/868156458


In [59]:
torben_df.columns

Index(['id', 'type', 'start_date', 'start_h', 'name', 'x_min', 'x_mi',
       'x_minpm', 'x_mph', 'x_max_mph', 'total_gain', 'avg_hr', 'max_hr',
       'avg_cadence', 'city_start', 'distance_start_end', 'start_latlng',
       'end_latlng', 'elev_low', 'elev_high', 'athlete_count', 'avg_temp',
       'distance', 'elapsed_time', 'start_latitude', 'start_longitude',
       'timezone', 'x_date', 'x_elev', 'x_url'],
      dtype='object')

In [46]:
#create connection to sqlite database 
connection_string = "sqlite:///fitness_database.sqlite"
engine = create_engine(connection_string)

In [47]:
#create tables in database
engine.execute("CREATE table garmin_ac (activity varchar, date varchar, title varchar,distance decimal,calories int,time varchar,avg_hr int, max_hr int, aerobic demical, avg_run int, max_run int, avg_pace varchar,best_pace varchar, elev_gain int, elev_loss int, avg_stride decimal, climb_time varchar, min_temp demical, best_lap varchar, laps int )")

In [48]:
garmin_df.to_sql(name='garmin_ac',con=engine, if_exists ='append',index=False)

In [49]:
garmin_data = engine.execute("select * from garmin_ac")
for record in garmin_data:
    print(record)

('Running', '10/29/2020 14:50', 'Park County Running', 1.02, 183, '0:39:41', 96, 127, 1, 49, 158, '39:03:00', '3:14', 276, 259, 0.83, '39:41:00', 64.4, '00:02.5', 2)
('Running', '10/29/2020 13:34', 'Park County Running', 0.72, 137, '0:31:13', 91, 117, 0.6, 40, 162, '43:33:00', '12:53', 105, 62, 0.91, '31:13:00', 68, '31:13.0', 1)
('Running', '10/27/2020 10:34', 'Grand County Running', 2.01, 224, '0:35:41', 110, 138, 1.7, 91, 250, '17:46', '5:22', 217, 207, 1, '35:41:00', 66.2, '00:06.0', 3)
('Running', '10/26/2020 15:12', 'San Juan County Running', 1.89, 259, '0:50:17', 104, 129, 0.9, 71, 225, '26:38:00', '13:49', 213, 184, 0.86, '50:17:00', 66.2, '22:53.0', 2)
('Running', '10/25/2020 11:27', 'Garfield County Running', 8.43, 1251, '3:01:06', 130, 178, 3.7, 97, 250, '21:30', '10:07', 1739, 1742, 0.77, '3:01:06', 60.8, '07:42.4', 9)
('Elliptical', '10/12/2020 23:17', 'Elliptical', 0, 238, '0:31:24', 119, 141, 2.2, 1, 123, '0', '0', 0, 0, 0, '31:24:00', 86, '31:23.7', 1)
('Elliptical', '1

In [69]:
engine.execute("CREATE table actorben (id int, type varchar,start_date varchar,start_h decimal,name varchar,x_min decimal,x_mi decimal,x_minpm decimal,x_mph decimal,x_max_mph,total_gain decimal,avg_hr decimal,max_hr decimal,avg_cadence decimal,city_start varchar,distance_start_end decimal,start_latlng varchar,end_latlng varchar,elev_low decimal,elev_high decimal, athlete_count int, avg_temp int, distance decimal,elapsed_time int, start_latitude decimal, start_longitude decimal, timezone varchar, x_date varchar, x_elev decimal, x_url varchar)")

In [70]:
torben_df.to_sql(name='actorben',con=engine,if_exists ='append',index = False)

In [72]:
torben_data = engine.execute("select * from actorben")
for record in torben_data:
    print(record)

(868156484, 'Run', '15.01.2017 06:17:41', 6.3, 'Morning Run', 248.2, 26.51, 9.36, 6.41, 9.17, 217.6, None, None, 80.8, 'NA-US-CA-Carlsbad', 0.2, '[33.176906, -117.335854]', '[33.177626, -117.337364]', -7.1, 97.1, 29, None, 42663.3, 14892, 33.176906, -117.335854, '(GMT-08:00) America/Los_Angeles', '2017-01-15', 0.51, 'https://www.strava.com/activities/868156484')
(868156415, 'Run', '17.01.2017 18:50:18', 18.8, 'Evening Run', 16.1, 2.023, 7.95, 7.55, 11.41, 0, None, None, 84.5, 'NA-US-CA-Coronado', 0.1, '[32.720301, -117.150205]', '[32.721214, -117.150103]', 40.7, 44.6, 30, None, 3256.2, 1300, 32.720301, -117.150205, '(GMT-08:00) America/Los_Angeles', '2017-01-17', None, 'https://www.strava.com/activities/868156415')
(868156459, 'Run', '19.01.2017 06:35:12', 6.6, 'Morning Run', 87, 10.091000000000001, 8.62, 6.96, 9.62, 137.6, None, None, 84, 'NA-US-CA-Santee', 0, '[32.853794, -116.960111]', '[32.853645, -116.960532]', 106, 201.5, 1, None, 16240.6, 5271, 32.853794, -116.96011100000001, '(